# METR 3123 HW \#3 – Due Monday March 21 at 9 PM.


### This is the jupyter notebook for our 3rd homework assignment.

---

### About our programming framework

Jupyter is a web-based interactive coding framework that many use as their primary programming platform, because its block formatting caters very well to debugging, while the blocks can be a mix of text (like this one), code, and figures.

Please watch the first 3 minutes of this great [tutorial](https://www.youtube.com/watch?v=HW29067qVWk) on jupyter as an intro. While the remainder of that video covers how to install jupyter on your own machine, *you will not need to install anything for our programming assignments.* We are using this online, fully self-contained approach called **Binder**, which removes the need for installing any software - rather, we can use this framework to modify preexisting python code for assignments.

Through this website, you are viewing a newly created **Binder**, which is a carbon copy of our [class Github repository](https://github.com/jhruppert/OU-Dynamics-3123). ***SAVE YOUR BINDER URL IN ORDER TO RETURN TO YOUR WORK.*** What you modify here will not change anything in our class github repository, but you can return to your work at a later time using the same Binder URL. If something goes wrong, you can always click the binder link in our github page again to start a new one.

___

### The Assignment

We will read in and analyze ERA5 reanalysis data from [Hurricane Sandy (2012)](https://en.wikipedia.org/wiki/Hurricane_Sandy), which during its landfall in New Jersey interacted with a phenomenal upper-level trough/ridge pattern.

The code blocks below are already set up to read in the data and plot horizontal maps of wind and geopotential height.

**What you will do:** you will modify and add to this code to produce horizontal maps for the 300-hPa level of:
1. Geostrophic wind
2. Ageostrophic wind
3. Divergence

Python functions of interest (google is your friend): "gradient"

To run the code, click on a code block to highlight it, then run it by pressing the Play button in the toolbar (or learn some [keyboard shortcuts](https://gist.github.com/discdiver/9e00618756d120a8c9fa344ac1c375ac)). Do the same thing progressing down the notebook in sequence. In this way, any code error will usually be contained to whichever block you're modifying.

---

This block loads up necessary packages:

In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import cartopy

The below command lists the contents of the data directory (7 netcdf files):

In [ ]:
!ls data/sandy/

Each file is a separate pressure level (the numbers in the filename are hPa), containing the following:
- 16 time steps (dt = 6 h), starting at 0000 UTC 27 Oct 2012
- With the variables:
    - "u" - zonal wind (m/s)
    - "v" - meridional ...
    - "w" - vertical ...
    - "t" - temperature (K)
    - "z" - geopotential ($m^2/s^2$)

In the below blocks, you'll find variables that you can modify to change the desired pressure level (changes the input file) and time step to plot.

In [ ]:
pressure = '300' # select the pressure level to plot
nc = Dataset('data/sandy/ERA5-sandy-pl_'+pressure+'.nc') # this opens the netcdf file

In [ ]:
# Read in some essentials

itime = 10 # choose index of time

# horizontal wind components
u = nc.variables['u'][itime, :, :] # result is 2d array (lat, lon), m/s
v = nc.variables['v'][itime, :, :]
# wind speed in m/s
spd = np.sqrt(u**2 + v**2)

# geopotential
phi = nc.variables['z'][itime, :, :] # m2/s2
# geopotential height (divide by gravity constant)
g=9.81 # m/s^2
geoz = phi / g / 10 # m2/s2 --> m --> dam

# make 2d lat, lon grid to match u, v size
lat, lon = np.meshgrid(nc.variables['latitude'][:], nc.variables['longitude'][:], indexing='ij')

In [ ]:
# New analysis



In [ ]:
# select plotting area
plt_area=[-120, -50, 10, 70] # W,E,S,N

spacing=10 #barbspacing (smaller if zoomed in)

# create figure
fig = plt.figure(figsize=(20,16))
ax = fig.add_subplot(111,projection=cartopy.crs.PlateCarree())
ax.set_title(pressure + ' hPa Winds', fontsize=20)

# fill contour for wind speed (jet streaks)
im = ax.contourf(lon, lat, spd, np.arange(30, 70, 10), cmap='GnBu', alpha=0.6, \
                 extend='max',projection=cartopy.crs.PlateCarree(), zorder=2)
cbar = plt.colorbar(im, ax=ax); cbar.ax.set_ylabel('m s$^{-1}$')

# line contour for geopotential height
im = ax.contour(lon, lat, geoz, np.arange(800, 1000, 10), colors='black', projection=cartopy.crs.PlateCarree(), zorder=2)

# wind barbs
mps_to_kts=1.94384 # conversion factor from m/s to knots for barbs
ax.barbs(lon[::spacing,::spacing], lat[::spacing,::spacing], \
         u[::spacing,::spacing]*mps_to_kts, v[::spacing,::spacing]*mps_to_kts, zorder=2)

# add map features
ax.add_feature(cartopy.feature.LAND,facecolor="lightgray") #land color
# ax.add_feature(cartopy.feature.OCEAN) #ocean color
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.STATES)
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Zoom into selected area (comment this out to plot entire available area)
ax.set_extent(plt_area)

plt.show()